# !! 라이브러리 geopadnas 설치가 우선시 되어야함!
conda가 아닌 환경에서 geopandas설치는 다음과 같은 라이브러리들이 준비되어야함.<br>
- 윈도우의 경우 pipwin을 통해 설치하여야한다
<p>pip install wheel<br>
pip install pipwin<br>
pipwin install pandas<br>
pipwin install shapely<br>
pipwin install gdal<br>
pipwin install fiona<br>
pipwin install pyproj<br>
pipwin install six<br>
pipwin install rtree<br>
pipwin install geopandas</p>

- 윈도우가 아니고 conda를 사용할 경우<br>
conda install --channel conda-forge geopandas<br>

- 윈도우가 아닌경우
<p>pip install wheel<br>
pip install pandas<br>
pip install shapely<br>
pip install gdal<br>
pip install fiona<br>
pip install six<br>
pip install pyproj<br>
pip install six<br>
pip install rtree<br>
pip install geopandas</p>

<strong>
위 과정을 거쳐 geopandas를 설치하여야함.</strong>


## Windows python환경 (windows가 아닌경우 위 참조)

In [ ]:
!pip install wheel

In [ ]:
!pip install pipwin

In [ ]:
!pipwin install pandas

In [ ]:
!pipwin install shapely

In [ ]:
!pipwin install gdal

In [ ]:
!pipwin install fiona

In [ ]:
!pipwin install pyproj

In [ ]:
!pipwin install six

In [ ]:
!pipwin install rtree

In [ ]:
!pipwin install geopandas

# conda 환경

In [ ]:
conda install --channel conda-forge geopandas

# 3. 구 비율 데이터 만들기 

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

# 사용할 util용 Code 로딩

In [ ]:
code = pd.read_csv('../Data/processing_data/2nd/busan_code.csv')

In [ ]:
sig_code = code[code['DONG'].isna()].reset_index(drop=True)

In [ ]:
sig_code = sig_code.drop(columns='DONG')

# 펌프작업

In [ ]:
pump_data1 = pd.read_csv('../data/rawdata/busan_pump_data.csv', encoding='euc-kr')
pump_data2 = gpd.read_file('../Data/rawdata/FL_PUMP/FL_PUMP_SP.shp')
pump_data1.iloc[29:31,0:1] = '금정구'
pump_data1.iloc[26:27,0:1] = '연제구'
code['CODE'] = code['CODE'].apply(str)
code
pump_data2 = pd.merge(code, pump_data2, left_on='CODE', right_on='ADM_CD', how='inner')
pump_data2.shape
pump_data2 = pump_data2.groupby(by='ZONE').count()['CODE']
pump_data2.sum()
pump_data1 = pump_data1.groupby(by='구군').count()['배수펌프장명']
pump_data = pump_data1.add(pump_data2,fill_value=0)
pump_data = pump_data.reset_index()
pump_data.rename(columns={'index':'ZONE',0:'PUMP_COUNT'},inplace=True)
pump_data['PUMP_COUNT'] = pump_data['PUMP_COUNT'].astype(int)
PIH_merge = pd.merge(pump_data,sig_code,on=['ZONE'],how='inner')

# 불투수면 작업

- 전국 데이터에서 부산 데이터를 뽑아냄 불투수면의 경우 법정동코드로 되어 있고 
-  부산의 법정동코드는 21로 시작하는데 부산의 '구' 데이터만 가져온다.

In [ ]:
imper = pd.read_excel('../data/rawdata/imper_ratio_data.xlsx')
imper['코드'] = imper['코드'].apply(str)
imper = imper[imper['코드'].str.startswith('21')].reset_index(drop=True)
imper = imper.drop(index=0).reset_index(drop=True)
imper = imper.drop(columns=['코드','기준연도','불투수면 면적(㎢)'])
imper.rename(columns={'시도시군구':'ZONE', '불투수면 비율(%)':'IMP_SUR_RATIO',\
                              '행정구역면적(㎢)':'ZONE_AREA'}, inplace=True)
imper['ZONE_AREA'] = round(imper['ZONE_AREA'],2) * 10000000
imper['IMP_SUR_RATIO'] = round(imper['IMP_SUR_RATIO'],2)

imper.sort_values('ZONE',inplace=True, ascending=False, ignore_index=True)


In [ ]:
PIH_merge = pd.merge(PIH_merge,imper, on='ZONE', how='outer')

In [ ]:
PIH_merge

In [ ]:
PIH_merge.fillna(0,inplace=True)
PIH_merge['PUMP_RATIO'] = PIH_merge['PUMP_COUNT'] * 10 / PIH_merge['ZONE_AREA']
PIH_merge.drop(columns='PUMP_COUNT', inplace=True)

# 침수빈도

In [ ]:
df = pd.read_csv("../Data/processing_data/1st/busan_flood_geo.csv")
df1 = df.copy()
a = df1[["ZONE","F_BEGIN_DE"]]
b = a.copy()
b.drop_duplicates(["ZONE","F_BEGIN_DE"],inplace=True)
b.astype({"ZONE":str})

# 가중치 구하기
zone = ['연제구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구',
       '금정구', '강서구', '수영구', '사상구', '기장군']
count = [4,1,1,2,1,1,1,1,3,5,1,3,4,1,1]
w = pd.DataFrame({"ZONE":zone,"COUNT":count})
bins = pd.DataFrame({"bin":w.groupby("COUNT").size()})
bins["percentage"]=round((bins["bin"]/bins["bin"].sum())*100,2)
bins["COUNT1"]=[1,2,3,4,5]
F_bin = pd.merge(w,bins,how="inner",left_on="COUNT",right_on="COUNT1")
F_bin.drop(["COUNT1","bin"],axis=1,inplace=True)
F_bin.rename(columns={"COUNT":"F_GRADE","percentage":"F_PERCENTAGE"},inplace=True)

In [ ]:
F_bin.info()

In [ ]:
F_bin
F_bin["F_WEIGHT"] = float(1.0)/F_bin["F_PERCENTAGE"]
F_bin

In [ ]:
PIH_merge = pd.merge(PIH_merge, F_bin, on='ZONE', how='left')
PIH_merge = PIH_merge.fillna(0)

In [ ]:
PIH_merge = PIH_merge.drop(columns='CODE')
PIH_merge = pd.merge(PIH_merge, sig_code, on='ZONE', how='left')

In [ ]:
PIH_merge

In [ ]:
PIH_merge.to_csv('../Data/processing_data/2nd/sig_info.csv',encoding='utf-8', index=False)

# 불투수면 가중치 계산

In [ ]:
imp_weight = PIH_merge['IMP_SUR_RATIO'].copy()

In [ ]:
for idx,col in enumerate(PIH_merge['IMP_SUR_RATIO']):
    if col <= 15:
        imp_weight[idx] = col / 6.66
    elif col > 15 and col <= 25:
        imp_weight[idx] = col / (20 * 2)
    elif col > 25 and col <= 40:
        imp_weight[idx] = col / (13.33 * 3)
    elif col > 40 and col <= 55:
        imp_weight[idx] = col / (33.33 * 4)
    elif col > 55:
        imp_weight[idx] = col / (26.67 * 5)

In [ ]:
PIH_merge['Impervious_Surface_Weight'] = imp_weight

In [ ]:
PIH_merge = PIH_merge[['CODE','ZONE','PUMP_RATIO','F_WEIGHT','Impervious_Surface_Weight' ]]

In [ ]:
PIH_merge.to_csv('../Data/processing_data/2nd/sig_info_weight.csv', encoding='utf-8', index=False)

In [ ]:
PIH_merge.to_csv('../Data/final_data/DB/sig_info_weight.csv', encoding='utf-8', index=False)